In [ ]:
import pandas as pd
import re

# CSV 파일 불러오기
df = pd.read_csv("3월 스미싱 탐지내역.csv")

In [ ]:
df[["Keyword","SMS"]].head()

,Keyword,SMS
0,공공기관,[경찰청교통민원]도로위반안내문자발송.확인바람:
1,기타,[국외발신]T？e？Ie？gr？am？？？？？？？？책？？？？？. 보？？？？？？증 완？？...
2,공공기관,[국외발신][경찰청교통민원]도로위반안내문자발송.확인바람:
3,공공기관,[국제발신][경찰청교통민원]도로위반안내문자발송.확인바람: [M1]
4,기타,고객님의 카????계정이 규???위반으로 공식 ???페이지 인???이 취소되었습니다


In [ ]:
print("중복 제거 전:", len(df))

#중복 제거

df = df.drop_duplicates(subset='SMS', keep='first').reset_index(drop=True)

print("중복 제거 후:", len(df))


df.to_csv("sms_drop_duplicates.csv", index=False)

중복 제거 전: 9285
중복 제거 후: 2841


In [ ]:
#df를 중복제거한 파일로 변경
df = pd.read_csv("sms_drop_duplicates.csv")

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-sz3pprk0
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-sz3pprk0
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286920 sha256=1c63dc6b863033a07691b1ebc8a5bfd4e864d412313981841f4bc4a1ff2d1c1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-xi9ams9k/wheels/1f/3f/64/6d5b2c9ba9cd5aa624676868e8ae8ec684672ce1bcbe20add8
Successfully built pykospacing


In [ ]:
import pandas as pd
import re
from pykospacing import Spacing

spacing = Spacing()

def full_preprocess(text):
    text = str(text)


    # 불필요한 토큰/괄호 제거
    text = text.replace("[M1]", "").replace("[FW]", "").replace("[fw]","")
    text = text.replace("{", "[").replace("}", "]")
    text = text.replace("【", "[").replace("】", "]")
    text = text.replace("<", "[").replace(">", "]")

    # 국제발신, 국외발신 처리
    #text = re.sub(r"\[(국제발신|국외발신)\]", "", text)

    # URL
    text = re.sub(r"(\(\s*\)|\<\s*\>|\[\s*\]|【\s*】)", "[URL]", text)
    text = re.sub(r"[:;]\s*$", "[URL]", text)

    #URL 토큰 중복 방지
    text = re.sub(r"(\[URL\]){2,}", "[URL]", text)

    # 시간
    text = re.sub(r"\d{1,2}시\s?\d{1,2}분(경)?", "[TIME]", text)
    text = re.sub(r"\d{1,2}시(반)?(경)?", "[TIME]", text)
    text = re.sub(r"\d{1,2}:\d{1,2}", "[TIME]", text)
    text = re.sub(r"(오전|오후|저녁|새벽)\s?\d{1,2}시간?", "[TIME]", text)

    # 단위
    text = re.sub(r"\d+(건|회|차)", "[COUNT]", text)
    text = re.sub(r"\d{1,3}(,\d{3})*(만원|원)", "[PRICE]", text)
    text = re.sub(r"\d+(만원|원)", "[PRICE]", text)

    # 전화번호 080 한정
    text = re.sub(r"080[-\s]?\d{3,4}[-\s]?\d{4}", "[PHONE]", text)

    # 날짜
    text = re.sub(r"\d{4}년\s?\d{1,2}월", "[DATE]", text)
    text = re.sub(r"\d{1,2}월\s?\d{1,2}일", "[DATE]", text)
    text = re.sub(r"\d{1,2}월", "[DATE]", text)
    text = re.sub(r"\d{1,2}일", "[DATE]", text)
    text = re.sub(r"\d{4}/\d{2}/\d{2}", "[DATE]", text)
    text = re.sub(r"\d{4}\.\d{2}\.\d{2}", "[DATE]", text)
    text = re.sub(r"\d{4}-\d{2}-\d{2}", "[DATE]", text)
    text = re.sub(r"\d{1,2}/\d{1,2}", "[DATE]", text)

    # 특수문자 & 중복 문자 정리
    text = re.sub(r"(ㅇ|ㅋ|ㅎ|!|\?|\?){2,}", r"\1", text)
    text = re.sub(r"[^\s\w가-힣[]]", "", text)
    text = re.sub(r"[\.\,\_\~\=\|\^\•\·\?\？!\<\>\(\)\-\\*\&\%\$+\:\;\'\"\/]", "", text)

    # 0000 제거
    text = re.sub(r"0{4}", "", text)

    # 남은 모든 숫자 NUM처리
    # text = re.sub(r"\d+", "[NUM]", text)

    # 5자리 이상 숫자 처리
    text = re.sub(r"\d{5,}","[NUM]",text)

    # 공백 제거 → 띄어쓰기 보정
    text = re.sub(r"\s+", "", text)
    spaced = spacing(text)
    spaced = re.sub(r"\s+", " ", spaced).strip()


    return spaced



# CSV read
df = pd.read_csv("sms_drop_duplicates.csv")

# 중복 제거
df = df.drop_duplicates(subset="SMS", keep="first").reset_index(drop=True)

# 전처리 적용
df['Result'] = df['SMS'].apply(full_preprocess)

# 저장
df.to_csv("sms_cleaned_final.csv", index=False)
print("✅ 최종 전처리 CSV 저장 완료: sms_cleaned_final.csv")

✅ 최종 전처리 CSV 저장 완료: sms_cleaned_final.csv
